In [322]:
##Packages
import inference
import os
import shutil
import numpy as np
from collections import Counter
import supervision as sv
from ultralytics import YOLO

In [335]:
def compute_iou(box1, box2):
    # Compute intersection coordinates
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    # Compute area of intersection
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    
    # Compute area of both bounding boxes
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    # Compute IoU
    union_area = box1_area + box2_area - inter_area
    iou = inter_area / union_area
    return iou

def filter_detections(detection):
    boxes = detection[0].boxes.xyxy.tolist()
    confidences = detection[0].boxes.conf.tolist()
    class_ids = detection[0].boxes.cls.int().tolist()
    
    num_boxes = len(boxes)
    to_keep = [True] * num_boxes

    for i in range(num_boxes):
        if not to_keep[i]:
            continue
        for j in range(i + 1, num_boxes):
            if compute_iou(boxes[i], boxes[j]) > 0.90:
                if confidences[i] < confidences[j]:
                    to_keep[i] = False
                else:
                    to_keep[j] = False

    # Filter the boxes based on the to_keep list
    filtered_boxes = [box for box, keep in zip(boxes, to_keep) if keep]
    filtered_confidences = [box for box, keep in zip(confidences, to_keep) if keep]
    filtered_class_ids = [box for box, keep in zip(class_ids, to_keep) if keep]

    class_names = [detection[0].names[int(class_id)] for class_id in class_indices]
    filtered_class_names = [box for box, keep in zip(class_names, to_keep) if keep]
    
    return {
        #Don't need most of this
        'xyxy': filtered_boxes,
        'confidence': filtered_confidences,
        'class_id': filtered_class_ids,
        'class_name': filtered_class_names
    }

def filter_detections2(detection):
    boxes = detection.xyxy
    confidences = detection.confidence
    class_ids = detection.class_id
    
    num_boxes = len(boxes)
    to_keep = [True] * num_boxes

    for i in range(num_boxes):
        if not to_keep[i]:
            continue
        for j in range(i + 1, num_boxes):
            if compute_iou(boxes[i], boxes[j]) > 0.90:
                if confidences[i] < confidences[j]:
                    to_keep[i] = False
                else:
                    to_keep[j] = False

    # Filter the boxes based on the to_keep list
    filtered_boxes = boxes[to_keep]
    filtered_confidences = confidences[to_keep]
    filtered_class_ids = class_ids[to_keep]

    return {
        #Don't need most of this
        #'xyxy': filtered_boxes,
        #'confidence': filtered_confidences,
        #'class_id': filtered_class_ids,
        'class_name': detection['class_name'][to_keep]
    }

In [336]:
def get_folder_name(predictions):
    class_indices = results[0].boxes.cls.int().tolist()
    # Get the names using the names mapping
    class_names = [results[0].names[int(class_id)] for class_id in class_indices]
    
    class_counts = Counter(class_names)
    #print(class_counts)
    # If there are no class names, use "no_dets"
    if not class_counts:
        return "1unknown-deer"
    
    # Create a list of class names with counts, sorted alphabetically by class name
    folder_parts = [f"{count}{class_name}" for class_name, count in sorted(class_counts.items())]

    # Join parts with underscores
    folder_name = "_".join(folder_parts)

    if any(word in folder_name.lower() for word in ["collar", "eartag", "cliptag"]):
        return "tagged"

    if folder_name == "1fawn_1male-deer":
        folder_name = "1Female deer_1fawn"
    
    return folder_name

def get_folder_name2(predictions):
    class_counts = Counter(predictions['class_name'])
    #print(class_counts)
    # If there are no class names, use "no_dets"
    if not class_counts:
        return "1unknown-deer"
    
    # Create a list of class names with counts, sorted alphabetically by class name
    folder_parts = [f"{count}{class_name}" for class_name, count in sorted(class_counts.items())]

    # Join parts with underscores
    folder_name = "_".join(folder_parts)

    if any(word in folder_name.lower() for word in ["collar", "eartag", "cliptag"]):
        return "tagged"

    if folder_name == "1fawn_1male-deer":
        folder_name = "1Female deer_1fawn"
    
    return folder_name

In [340]:
## Locations
#Grab models, specify where your images are stored and where you want results folders to go
clipmod = YOLO('/Users/heather/runs/detect/yolov8n_earclips/weights/best.pt')
earmod = YOLO('/Users/heather/runs/detect/yolov8n_eartags/weights/best.pt')
deermod = YOLO('/Users/heather/runs/detect/yolov8n_deer2/weights/best.pt')
#model = inference.get_model("deer2-sxa9n/12", api_key = MYKEYHERE) #maybe worse
#model.confidence = 30
image_folder = '/Users/heather/Desktop/U_Georgia/Chandler_Meetings/heather/CWD_Postdoc/Billionphotos/GR56_07.06.24/MDoutput_07_06_24/animals/deer'

destination_folder = image_folder
os.makedirs(destination_folder, exist_ok=True)
#initialize empty list:
results_list = []

In [342]:
image_files = [filename for filename in os.listdir(image_folder) if filename.lower().endswith(('.png', '.jpg', '.jpeg'))]
# Get the total number of images
total_images = len(image_files)
for index, filename in enumerate(image_files, start=1):
    # Check if the file is an image (you might want to adjust the extensions)
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        print(f"Processing image {index}/{total_images}")
        # Construct the full path to the image
        image_path = os.path.join(image_folder, filename)
        # Process the image
        #First, use earclip model:
        results = clipmod.predict(image_path, conf= .75, verbose = False)
        if results[0].boxes.conf.numel() == 0:
            #print('no clips')
            #if no results from clip mod, use ear mod
            results = earmod.predict(image_path, conf= .5, verbose = False) 
            if results[0].boxes.conf.numel() == 0:
                #print('no eartags either')
                results = deermod.predict(image_path, conf= .45, verbose = False) #untagged animals
                filtered = filter_detections(results)
                folder_name = get_folder_name(filtered_detection)
                
                #results = model.infer(image=image_path)
                #predictions = results[0].predictions if results[0].predictions else []
                #detections = sv.Detections.from_inference(results[0])
                #filtered_detection = filter_detections2(detections)
                # Get the class names folder name
                #folder_name = get_folder_name2(filtered_detection)
            else: 
                folder_name = 'tagged' #ear tags
        else:
            folder_name = 'tagged' #ear clips


        #print(folder_name)
        #save it:
        # Define the destination path
        dest_folder = os.path.join(destination_folder, folder_name)
        # Create the folder if it doesn't exist
        os.makedirs(dest_folder, exist_ok=True)
        # Define the source and destination paths
        src_path = os.path.join(image_folder, filename)
        dest_path = os.path.join(dest_folder, filename)
        # Move the file
        shutil.move(src_path, dest_path)
        #print(f"Moved {src_path} to {dest_path}")
print("Finished Batch")

Processing image 1/26
Processing image 2/26
Processing image 3/26
Processing image 4/26
Processing image 5/26
Processing image 6/26
Processing image 7/26
Processing image 8/26
Processing image 9/26
Processing image 10/26
Processing image 11/26
Processing image 12/26
Processing image 13/26
Processing image 14/26
Processing image 15/26
Processing image 16/26
Processing image 17/26
Processing image 18/26
Processing image 19/26
Processing image 20/26
Processing image 21/26
Processing image 22/26
Processing image 23/26
Processing image 24/26
Processing image 25/26
Processing image 26/26
Finished Batch
